In [2]:
import pandas as pd
import numpy as np

Create Stats Data

In [47]:
#Using matches_final.csv and merged_lineups_final.csv.csv
mfn = pd.read_csv('../../Data/Data Preparation/matches_final.csv')
lineup = pd.read_csv('../../Data/Data Preparation/merged_lineups_final.csv')
ppt = 11 #Player per team
statsCount = 36 #Number of stats used
lineup.head(5)

,team_id,fixture_id,player_id,player_name,number,position,date,club,lastname,firstname,...,FKAcc.,Penalties,Volleys,GKPositioning,GKDiving,GKHandling,GKKicking,GKReflexes,Overall,Potential
0,1,2864,1251,Adrian,13,G,2016-01-02,West Ham United,Adrian,Adrian,...,17,44,12,76,78,76,71,85,78,78
1,1,2864,1261,Aaron Cresswell,3,D,2016-01-02,West Ham United,Cresswell,Aaron,...,69,59,29,9,14,7,9,12,73,75
2,1,2864,336,James Tomkins,5,D,2016-01-02,West Ham United,Tomkins,James,...,31,43,28,9,11,8,6,14,73,74
3,1,2864,538,James Collins,19,D,2016-01-02,West Ham United,Collins,James,...,64,51,36,15,15,12,10,11,74,74
4,1,2864,1416,Angelo Ogbonna,21,D,2016-01-02,West Ham United,Ogbonna,Angelo,...,36,39,30,4,5,3,5,4,79,79


In [3]:
# Get fixtures' id list  
fixtures_df = pd.read_csv('../../Data/Data Preparation/fixtures_list.csv')
fixtures_list = fixtures_df['fixture_id'].tolist()

In [48]:
mfn = mfn[mfn['fixture_id'].isin(fixtures_list)]

In [49]:
numsLnup = ['team_id', 'fixture_id', 'Height', 'Weight', 'BallControl', 'Marking', 'Dribbling', 'SlideTackle',
 'StandTackle', 'Aggression', 'Reactions', 'Att.Position', 'Interceptions', 'Vision', 'Crossing', 'ShortPass', 'LongPass',
 'Acceleration', 'Stamina', 'Strength', 'Balance', 'SprintSpeed', 'Agility', 'Jumping', 'Heading', 'ShotPower', 'Finishing',
 'LongShots', 'Curve', 'FKAcc.', 'Penalties', 'Volleys', 'GKPositioning', 'GKDiving', 'GKHandling', 'GKKicking',
 'GKReflexes', 'Overall']
numsCols = lineup[numsLnup]
colsName = ['id','home_id','away_id']
for i in range(statsCount):
    colsName.append('gap_stat_' + str(i+1))

iptStats = ['Vision','LongPass','SprintSpeed','Acceleration','Finishing' ,'LongShots', 'Curve']
for i in iptStats:
    colsName.append('gap_max_'+ i)

FormStats = pd.DataFrame(columns=colsName)

for i in range(int(lineup.shape[0]/(2*ppt))):
    sum = 0
    for j in range(2*ppt):
        if numsCols['team_id'][i*22+ j] == mfn[mfn['fixture_id'] == numsCols.fixture_id.iloc[i*22+ j]]['home_id'].values[0]:
            sum = np.array(sum + numsCols.iloc[i*22+ j, 2:].values)
        else: 
            sum = np.array(sum - numsCols.iloc[i*22+ j, 2:].values)
    sum = sum/11

    for j in iptStats:
        gapMaxStat = 0
        for k in range(2):
            if numsCols['team_id'][i*22+ k*11] == mfn[mfn['fixture_id'] == numsCols.fixture_id.iloc[i*22]]['home_id'].values[0]:
                gapMaxStat = gapMaxStat + numsCols[j].iloc[i*22+ k*11: i*22+ (k+1)*11].max()
            else:
                gapMaxStat = gapMaxStat - numsCols[j].iloc[i*22+ k*11: i*22+ (k+1)*11].max()
        sum = np.append(sum, gapMaxStat)
    match_id = int(numsCols.fixture_id.iloc[i*22])
    home_id = int(mfn[mfn['fixture_id'] == numsCols.fixture_id.iloc[i*22]]['home_id'].values[0])
    away_id = int(mfn[mfn['fixture_id'] == numsCols.fixture_id.iloc[i*22]]['away_id'].values[0])
        
    ids = np.array([match_id, home_id, away_id])
    sum = np.concatenate((ids, sum), axis= None)
    FormStats.loc[i] = sum
FormStats[['id','home_id','away_id']] = FormStats[['id','home_id','away_id']].astype(int)
FormStats


,id,home_id,away_id,gap_stat_1,gap_stat_2,gap_stat_3,gap_stat_4,gap_stat_5,gap_stat_6,gap_stat_7,...,gap_stat_34,gap_stat_35,gap_stat_36,gap_max_Vision,gap_max_LongPass,gap_max_SprintSpeed,gap_max_Acceleration,gap_max_Finishing,gap_max_LongShots,gap_max_Curve
0,2864,1,8,2.090909,-0.363636,-6.000000,0.000000,-7.909091,-1.363636,-3.090909,...,0.363636,1.545455,-3.272727,-9.0,-1.0,5.0,3.0,-5.0,0.0,-10.0
1,2876,3,15,-2.181818,-2.636364,-0.727273,-5.363636,0.636364,-5.454545,-7.363636,...,0.000000,1.363636,-0.090909,-3.0,5.0,10.0,10.0,10.0,8.0,0.0
2,2886,42,52,-1.272727,-1.545455,-0.272727,-5.181818,-2.636364,-1.454545,-1.545455,...,1.454545,1.636364,2.454545,-2.0,1.0,1.0,3.0,5.0,4.0,17.0
3,2897,19,20,0.727273,-3.272727,3.181818,-2.363636,-0.363636,-1.545455,-1.909091,...,-1.909091,1.000000,5.727273,16.0,7.0,11.0,16.0,1.0,7.0,5.0
4,2908,33,65,-1.727273,-5.454545,-5.636364,-12.000000,-6.272727,-4.909091,-5.727273,...,-0.363636,0.272727,-3.818182,-2.0,-6.0,0.0,5.0,4.0,5.0,-6.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11426,18165753,3457,83,-0.818182,-2.090909,-7.363636,1.636364,-7.909091,-0.454545,2.181818,...,-1.818182,-1.272727,-7.454545,-1.0,-5.0,-8.0,-8.0,-4.0,1.0,1.0
11427,18165757,377,214,-3.454545,0.363636,-1.636364,-1.909091,-3.636364,-4.454545,-5.818182,...,-0.090909,1.727273,-1.818182,-6.0,-6.0,0.0,4.0,-4.0,-9.0,-5.0
11428,18165743,594,485,2.272727,1.454545,0.636364,3.545455,0.000000,0.545455,0.454545,...,-1.727273,0.090909,0.272727,1.0,-4.0,-2.0,-6.0,0.0,-6.0,-3.0
11429,302049,613,2930,-0.272727,-0.909091,-4.454545,-6.818182,-3.000000,-5.363636,-6.727273,...,0.636364,-1.090909,-4.545455,-8.0,-7.0,3.0,10.0,-3.0,-6.0,-3.0


Create Form-Rest Data

In [4]:
# Import data
matches_df = pd.read_csv('../../Data/Data Preparation/matches_final.csv')

In [5]:
# Convert date column to datetime
matches_df['date'] = pd.to_datetime(matches_df['date'], format="%Y-%m-%d")

In [6]:
matches_df.head(5)

,fixture_id,date,home_id,home_name,home_scored,home_logo,away_id,away_name,away_scored,away_logo
0,18243482,2011-08-13,2,Blackburn Rovers,1,https://cdn.sportmonks.com/images/soccer/teams...,29,Wolverhampton Wanderers,2,https://cdn.sportmonks.com/images/soccer/teams...
1,18243483,2011-08-13,11,Fulham,0,https://cdn.sportmonks.com/images/soccer/teams...,15,Aston Villa,0,https://cdn.sportmonks.com/images/soccer/teams...
2,18243484,2011-08-13,8,Liverpool,1,https://cdn.sportmonks.com/images/soccer/teams...,3,Sunderland,1,https://cdn.sportmonks.com/images/soccer/teams...
3,18243485,2011-08-13,47,Queens Park Rangers,0,https://cdn.sportmonks.com/images/soccer/teams...,16,Bolton Wanderers,4,https://cdn.sportmonks.com/images/soccer/teams...
4,18243486,2011-08-13,17,Wigan Athletic,1,https://cdn.sportmonks.com/images/soccer/teams...,33,Norwich City,1,https://cdn.sportmonks.com/images/soccer/teams...


In [7]:
# Drop useless column
matches_df = matches_df.drop(columns=['home_logo', 'away_logo'])

In [8]:
def get_result(row):
    if row['home_scored'] > row['away_scored']:
        return 1
    elif row['home_scored'] < row['away_scored']:
        return -1
    else:
        return 0

In [9]:
# Add result column 
matches_df['result'] = matches_df.apply(get_result, axis=1)

In [10]:
# Add point's columns for both home and away team from result column
matches_df['home_points'] = matches_df['result'].map({0:1, 1:3, -1:0})
matches_df['away_points'] = matches_df['result'].map({0:1, 1:0, -1:3})

In [11]:
# Add conceded columns for both home and away team from result column
matches_df['home_conceded'] = matches_df['away_scored']
matches_df['away_conceded'] = matches_df['home_scored']

In [12]:
# Mask home teams 
home_df = matches_df.loc[:, ['home_id', 'date', 'home_points', 'home_scored', 'home_conceded']]
home_df = home_df.rename(columns={'home_id': 'team_id', 'home_points': 'team_points','home_scored': 'team_scored','home_conceded': 'team_conceded'})
# Mask away teams 
away_df = matches_df.loc[:, ['away_id', 'date', 'away_points', 'away_scored', 'away_conceded']]
away_df = away_df.rename(columns={'away_id': 'team_id', 'away_points': 'team_points','away_scored': 'team_scored','away_conceded': 'team_conceded'})
# Concatenate vertically mask_home and mask_away (avoided problem of using separate home and away IDs by using a team ID)
form_teams_df = pd.concat([home_df, away_df], ignore_index=True).sort_values('date').reset_index(drop=True)

In [13]:
# List of numbers of games to consider
list_number_games = [1, 3, 5, 10, 20]

# List of features
feats = ['_points', '_scored', '_conceded']

# Prefixes for team locations
team_location_prefixes = ['home', 'away']

for index, row in matches_df.iterrows():
    for prefix in team_location_prefixes:
        # Filter form_teams_df to get data for the same team and matches before the current match date
        same_team_df = form_teams_df[(form_teams_df['team_id'] == getattr(row, f'{prefix}_id')) & (form_teams_df['date'] < row.date)]
        
        # Sort the filtered DataFrame by date in descending order
        same_team_df = same_team_df.sort_values(by='date', ascending=False)

        # Calculate the rest days
        if not same_team_df.empty:
            matches_df.at[index, f'{prefix}_rest_days'] = (row.date - same_team_df.iloc[0].date).days
        else:
            matches_df.at[index, f'{prefix}_rest_days'] = 0

        for n in list_number_games:
            for feat in feats:
                # Calculate the mean values of last n games features
                matches_df.at[index, f'{prefix}{feat}_last_{n}_games'] = np.mean(same_team_df.loc[same_team_df['date'].nlargest(n=n, keep='last').index, f'team{feat}'])

# Remove outliers in rest days
matches_df.loc[:, 'home_rest_days'] = np.where(matches_df.loc[:, 'home_rest_days'] < 30, matches_df.loc[:, 'home_rest_days'], 30)
matches_df.loc[:, 'away_rest_days'] = np.where(matches_df.loc[:, 'away_rest_days'] < 30, matches_df.loc[:, 'away_rest_days'], 30)

# Fill null values by 0
matches_df[matches_df.columns[matches_df.columns.get_loc('home_points_last_1_games'):]] = matches_df[matches_df.columns[matches_df.columns.get_loc('home_points_last_1_games'):]].fillna(0)

In [26]:
# Filter by fixtures' id list
form_rest_df = matches_df[matches_df['fixture_id'].isin(fixtures_list)]

In [39]:
form_rest_df.head()

,fixture_id,date,home_id,home_name,home_scored,away_id,away_name,away_scored,result,home_points,...,conceded_last_3_games_diff,points_last_5_games_diff,scored_last_5_games_diff,conceded_last_5_games_diff,points_last_10_games_diff,scored_last_10_games_diff,conceded_last_10_games_diff,points_last_20_games_diff,scored_last_20_games_diff,conceded_last_20_games_diff
1710,2864,2016-01-02,1,West Ham United,2,8,Liverpool,0,1,3,...,-0.333333,0.0,-0.2,-1.0,-0.5,-0.6,-0.1,-0.050000,0.250000,-0.150000
1711,2876,2016-01-02,3,Sunderland,3,15,Aston Villa,1,1,3,...,1.333333,-0.6,0.0,1.0,0.5,0.4,0.0,0.200000,0.250000,0.300000
1712,2886,2016-01-02,42,Leicester City,0,52,AFC Bournemouth,0,0,1,...,0.000000,0.0,0.6,0.0,1.1,0.7,-0.9,1.047368,0.942105,-0.489474
1713,2897,2016-01-02,19,Arsenal,1,20,Newcastle United,0,1,3,...,0.666667,1.0,0.8,0.4,0.9,1.0,-0.4,1.100000,0.800000,-0.750000
1714,2908,2016-01-02,33,Norwich City,1,65,Southampton,0,1,3,...,0.000000,0.6,-0.2,0.2,0.0,-0.3,0.1,-0.200000,-0.200000,0.450000


In [28]:
# Create features_diff columns
features_list = ['rest_days', 'points_last_1_games', 'scored_last_1_games', 'conceded_last_1_games', 'points_last_3_games', 'scored_last_3_games', 'conceded_last_3_games', 'points_last_5_games', 'scored_last_5_games', 'conceded_last_5_games', 'points_last_10_games', 'scored_last_10_games', 'conceded_last_10_games', 'points_last_20_games', 'scored_last_20_games', 'conceded_last_20_games']

for feature in features_list:
    form_rest_df.loc[:, feature + '_diff'] = form_rest_df['home_' + feature].copy() - form_rest_df['away_' + feature].copy()


In [60]:
diff_columns_list = form_rest_df.filter(like='_diff').columns.tolist()
form_rest_df = form_rest_df[['fixture_id'] + diff_columns_list + ['result']]
form_rest_df.head(5)

,fixture_id,rest_days_diff,points_last_1_games_diff,scored_last_1_games_diff,conceded_last_1_games_diff,points_last_3_games_diff,scored_last_3_games_diff,conceded_last_3_games_diff,points_last_5_games_diff,scored_last_5_games_diff,conceded_last_5_games_diff,points_last_10_games_diff,scored_last_10_games_diff,conceded_last_10_games_diff,points_last_20_games_diff,scored_last_20_games_diff,conceded_last_20_games_diff,result
1710,2864,2.0,0.0,1.0,1.0,-0.333333,0.333333,-0.333333,0.0,-0.2,-1.0,-0.5,-0.6,-0.1,-0.050000,0.250000,-0.150000,1
1711,2876,-2.0,0.0,0.0,-1.0,-0.666667,0.000000,1.333333,-0.6,0.0,1.0,0.5,0.4,0.0,0.200000,0.250000,0.300000,1
1712,2886,-1.0,1.0,0.0,-2.0,0.000000,0.333333,0.000000,0.0,0.6,0.0,1.1,0.7,-0.9,1.047368,0.942105,-0.489474,0
1713,2897,0.0,3.0,2.0,-1.0,1.666667,1.000000,0.666667,1.0,0.8,0.4,0.9,1.0,-0.4,1.100000,0.800000,-0.750000,1
1714,2908,0.0,3.0,1.0,-2.0,1.000000,-0.333333,0.000000,0.6,-0.2,0.2,0.0,-0.3,0.1,-0.200000,-0.200000,0.450000,1


Create Final Data from Stats and Form-Rest Data

In [63]:
final_df = FormStats.merge(right=form_rest_df, left_on='id', right_on='fixture_id')

In [64]:
final_df.to_csv('../../Data/Final Data Modelling/final_data.csv', index=False)